In [57]:
import re
from open_spiel.python.algorithms import cfr, outcome_sampling_mccfr, expected_game_score, exploitability, get_all_states_with_policy
import pyspiel
from open_spiel.python.algorithms.cfr import CFRSolver
import pandas as pd
import pickle
import cProfile, pstats

In [58]:
# GAME_PATH = '/apps/open_spiel/sats/python/odd_licenses.json'
# GAME_PATH = 'very_small_sats_complete_symmetric.json'
# GAME_PATH = '/apps/open_spiel/sats/python/mixed.json'

In [59]:
def setup_game(GAME_NAME):
    GAME_PATH = f'/apps/open_spiel/sats/python/{GAME_NAME}.json'
    PYSATS = '/apps/sats/python'
    !python {PYSATS}/pysats.py --config_file {PYSATS}/{GAME_NAME}.yml --seed 777 --output_file {PYSATS}/{GAME_NAME}.json
    # GAME_NAME = f'{PYSATS}/{GAME_NAME}'
    !mv {GAME_PATH} /apps/open_spiel/configs/{GAME_NAME}.json

In [61]:
GAME_NAMES = ['may9']
solver = 'cfr'
n_seeds = 1
seeds = range(1234, 1234 + n_seeds)
iters = 100
for GAME_NAME in GAME_NAMES:
    setup_game(GAME_NAME)
    for seed in seeds:
        command = f'/apps/cluster_open_spiel/open_spiel/python/examples/ubc_mccfr_cpp_example.py --filename {GAME_NAME}.json --iterations {iters} --output results/{GAME_NAME}_{seed} --game python_clock_auction --python true --solver {solver}  --seed {seed}'
        print(command)
        !python {command}

{'licenses': [3], 'activity': [1], 'opening_price': [18], 'information_policy': 'show_demand', 'undersell_rule': 'undersell', 'increment': 1, 'fold_randomness': True, 'max_rounds': 5, 'agent_memory': 20, 'license_names': ['British Columbia'], 'players': [{'type': [{'value': [0, 60, 120, 120], 'value_format': 'full', 'budget': 120, 'prob': 1.0, 'name': 'normal', 'drop_out_heuristic': True}]}, {'type': [{'value': [0, 100, 250, 250], 'value_format': 'full', 'budget': 250, 'prob': 1.0, 'name': 'normal', 'drop_out_heuristic': True, 'straightforward': True}]}]}
/apps/cluster_open_spiel/open_spiel/python/examples/ubc_mccfr_cpp_example.py --filename may9.json --iterations 100 --output results/may9_1234 --game python_clock_auction --python true --solver cfr  --seed 1234
I0509 19:49:14.884173 140211714671104 clock_auction_parser.py:14] Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
I0509 19:49:14.884336 140211714671104 clock_auction_parser.py:18] CLO

In [ ]:
# def prof():
#     game = pyspiel.load_game("python_clock_auction", dict(filename='april6_t1.json'))
#     num_iterations = 500
#     cfr_solver = CFRSolver(game)
#     for i in range(num_iterations):
#         cfr_solver.evaluate_and_update_policy()


# profiler = cProfile.Profile()
# profiler.enable()
# prof()
# profiler.disable()

In [ ]:
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()


In [ ]:
game = pyspiel.load_game("python_clock_auction", dict(filename='april6_t1.json'))


def state_to_final_python(game, s):
    '''Convert a state into unique final outcomes (but not caring about bidding being different in the middle). i.e., the allocation and the types and the price are all the same'''
    info_state_string = f'Price: {s.posted_prices[-1]}\n'
    info_state_string += f'Final bids: {s.get_allocation()}\n'
    info_state_string += f'Round: {s.round}\n'
    return info_state_string


all_states = get_all_states_with_policy.get_all_info_states_with_policy(
    game,
    depth_limit=-1,
    include_terminals=True,
    policy=None,
    to_string=lambda s: state_to_final_python(game, s)
)


In [ ]:
game = pyspiel.load_game("python_clock_auction", dict(filename='april6_t1.json'))
solver = pickle.load(open('april6_t1/cfr_9.pkl', 'rb'))
policy = solver.average_policy()

all_states = get_all_states_with_policy.get_all_info_states_with_policy(
    game,
    depth_limit=-1,
    include_terminals=True,
    policy=policy,
    to_string=lambda s: state_to_final_python(game, s)
)


In [ ]:
game.new_initial_state().get_game()

In [ ]:
import pprofile
profiler = pprofile.Profile()
with profiler:
    prof()
profiler.dump_stats('output_april17.prof')

In [ ]:
game = pyspiel.load_game("python_clock_auction", dict(filename='april6_t1.json'))

In [ ]:
game.make_py_observer().string_from.cache_info()